# Анализ базы данных при помощи SQL

# 1.  Изучение данных

In [1]:
import pandas as pd
from sqlalchemy import create_engine, text

In [2]:
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

**Таблица books**

In [3]:
query = '''
            SELECT *  
            FROM books 
        '''
con = engine.connect()
books = pd.io.sql.read_sql(sql = text(query), con = con)

books

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


**Таблица authors**

In [4]:
query = '''
            SELECT *  
            FROM authors LIMIT 5
        '''
con = engine.connect()
authors = pd.io.sql.read_sql(sql = text(query), con = con)

authors

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**Таблица publishers**

In [5]:
query = '''
            SELECT *  
            FROM publishers LIMIT 5
        '''
con = engine.connect()
publishers = pd.io.sql.read_sql(sql = text(query), con = con)

publishers

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**Таблица ratings**

In [6]:
query = '''
            SELECT *  
            FROM ratings LIMIT 5
        '''
con = engine.connect()
ratings = pd.io.sql.read_sql(sql = text(query), con = con)

ratings

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**Таблица reviews**

In [7]:
query = '''
            SELECT *  
            FROM reviews LIMIT 5
        '''
con = engine.connect()
reviews = pd.io.sql.read_sql(sql = text(query), con = con)

reviews

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


# 2.  Ответим на вопросы

## 2.1 Посчитаем, сколько книг вышло после 1 января 2000 года?

In [8]:
query = ''' 
            SELECT COUNT(book_id) as books_count
            FROM books
            WHERE publication_date >= DATE('2000-01-01')
        '''
con = engine.connect()
pd.io.sql.read_sql(sql = text(query), con = con)

,books_count
0,821


После 1 января 2000 года было выпущено 819 книг.

## 2.2 Для каждой книги посчитаем количество обзоров и среднюю оценку

In [9]:
query = ''' 
            SELECT b.book_id,
                   b.title,
                   ROUND((one.avg_rating),2),
                   two.review_count
            FROM books as  b
            LEFT JOIN (SELECT book_id,
                              AVG(rating) as avg_rating
                 
                       FROM ratings
                       GROUP BY book_id) as one ON b.book_id=one.book_id
            
            LEFT JOIN (SELECT book_id,
                               COUNT(review_id) as review_count 
                               
                       FROM reviews 
                       GROUP BY book_id) as two ON b.book_id=two.book_id
            
        '''
con = engine.connect()
pd.io.sql.read_sql(sql = text(query), con = con)

,book_id,title,round,review_count
0,652,The Body in the Library (Miss Marple #3),4.50,2.0
1,273,Galápagos,4.50,2.0
2,51,A Tree Grows in Brooklyn,4.25,5.0
3,951,Undaunted Courage: The Pioneering First Missio...,4.00,2.0
4,839,The Prophet,4.29,4.0
...,...,...,...,...
995,64,Alice in Wonderland,4.23,4.0
996,55,A Woman of Substance (Emma Harte Saga #1),5.00,2.0
997,148,Christine,3.43,3.0
998,790,The Magicians' Guild (Black Magician Trilogy #1),3.50,2.0


Из данных  видно, что для большинства книг было написано несколько обзоров.

# 2.3 Определим издательство выпустившее наибольшее число книг толще 50 страниц

In [10]:
query = ''' 
            SELECT publisher, 
                   COUNT(book_id) as books_count 
            FROM books as b
            LEFT JOIN publishers as p ON b.publisher_id = p.publisher_id
            WHERE num_pages > 50
            GROUP BY publisher
            ORDER BY books_count DESC
            LIMIT 1            
        '''
con = engine.connect()
pd.io.sql.read_sql(sql = text(query), con = con)

,publisher,books_count
0,Penguin Books,42


Издательство "Penguin Books" выпустило 42 книги толще 50 страниц. 

# 2.4 Определим автора с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками

In [11]:
query = '''             
        SELECT author,
               ROUND(AVG(avg_raiting), 2) as avg_raiting, 
               SUM(rating_count) AS rate_count
        FROM    
        (SELECT author_id, 
                title, 
                AVG(rating) as avg_raiting, 
                COUNT(rating) as rating_count
        FROM books as b
        LEFT JOIN ratings as r ON b.book_id = r.book_id 
        GROUP BY title, author_id
        HAVING COUNT(rating_id) >= 50) as one
        LEFT JOIN authors as a ON one.author_id = a.author_id
        GROUP BY author
        ORDER BY avg_raiting DESC
        LIMIT 1                
        '''
con = engine.connect()
pd.io.sql.read_sql(sql = text(query), con = con)

,author,avg_raiting,rate_count
0,J.K. Rowling/Mary GrandPré,4.28,310.0


Наивысшая средняя оценка книг у Джоан Роулинг. Cерия книг о Гарри Поттере очень популярна.

# 2.5 Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [12]:
query = '''             
        SELECT ROUND(AVG(rev_counts)) as avg_rev
        FROM
            (SELECT COUNT(review_id) as rev_counts
            FROM
                (SELECT username 
                FROM ratings
                GROUP BY username
                HAVING COUNT(rating_id) > 48) as one
            LEFT JOIN reviews as r ON one.username = r.username
            GROUP BY one.username) as two
        '''
con = engine.connect()
pd.io.sql.read_sql(sql = text(query), con = con)

,avg_rev
0,24.0


Читатели, которые поставили больше 48 оценок, в среднем пишут по 24 обзора

# 3. Вывод

- В БД нового сервиса содержится информация о 1000 книг

- Количество книг, выпущенных после 1 января 2000 года, составляет 821 книгу.

- Посчитали количество обзоров , среднюю оценку для каждой из книг

- Издательство, которое выпустило наибольшее число книг толще 50 страниц -       Penguin Books с 42 книгами 

- Автор с самой высокой средней оценкой книг - это J.K. Rowling/Mary GrandPré     со средним рейтингом 4.28, составленным по 310 оценкам пользователей

- В среднем количество обзоров от пользователей, которые поставили больше 48     оценок - 24 обзора


 

